In [2]:
import pandas as pd
# Load real_data.txt
training_data = pd.read_csv(r"C:\Users\igriz\Documents\BOOTCAMP 2024\WEEK 7\DAY3\project-3-nlp-master\project-3-nlp-master\TRAINING_DATA.txt", delimiter="\t", header=None, names=["label", "spanish_sentence"])

# Display the first few rows
print(training_data.head())

   label                                   spanish_sentence
0      1  Cuando conocí a Janice en 2013 , una familia n...
1      0  Hwang habló en Sur de este año por Southwest M...
2      1  Usted podría pensar Katy Perry y Robert Pattin...
3      1  Cualquiera que haya volado los cielos del crea...
4      1  Bueno , este cantante tendrá un LARGO tiempo p...


In [3]:
data = training_data.reset_index(drop=True)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Reduce max features to 1000 (previously 5000)
tfidf = TfidfVectorizer(max_features=1000)

# Fit and transform the text data
X_tfidf = tfidf.fit_transform(data['spanish_sentence'])


In [5]:
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 5.7 MB/s eta 0:00:03
     ---------- ----------------------------- 3.4/12.8 MB 11.9 MB/s eta 0:00:01
     ----------------- ---------------------- 5.5/12.8 MB 11.6 MB/s eta 0:00:01
     -------------------- ------------------- 6.6/12.8 MB 11.2 MB/s eta 0:00:01
     ------------------------------- ------- 10.5/12.8 MB 11.5 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 11.6 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
import spacy

# Load a smaller GloVe model
nlp = spacy.load("en_core_web_sm")

# Function to get sentence embeddings
def get_glove_embeddings(texts):
    return [nlp(text).vector for text in texts]

X_glove = get_glove_embeddings(data['spanish_sentence'])


In [7]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

# SVM pipeline with fewer TF-IDF features
svm_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),  # Reduced from 5000 to 1000
    ('svm', SVC(kernel='linear', C=0.5))  # C=0.5 for faster training
])

svm_pipeline.fit(data['spanish_sentence'], data['label'])


Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=1000)),
                ('svm', SVC(C=0.5, kernel='linear'))])

In [8]:
from sklearn.ensemble import RandomForestClassifier

rf_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),  # Reduced features
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42))  # Only 10 trees
])

rf_pipeline.fit(data['spanish_sentence'], data['label'])


Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=1000)),
                ('rf',
                 RandomForestClassifier(n_estimators=10, random_state=42))])

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
# Tokenize sentences
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['spanish_sentence'])
X_seq = tokenizer.texts_to_sequences(data['spanish_sentence'])
X_pad = pad_sequences(X_seq, maxlen=100)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X_pad, data['label'], test_size=0.2, random_state=42)

# LSTM Model (Smaller & Faster)
model = Sequential([
    Embedding(input_dim=2000, output_dim=64),  # Smaller embedding
    LSTM(16, dropout=0.2, recurrent_dropout=0.2),  # Fewer units
    Dense(1, activation='sigmoid')
])

# Compile with Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train for only 3 epochs (instead of 5+)
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 18s 40ms/step - accuracy: 0.5031 - loss: 0.6936 - val_accuracy: 0.5183 - val_loss: 0.6925
Epoch 2/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - accuracy: 0.5681 - loss: 0.6834 - val_accuracy: 0.4838 - val_loss: 0.6885
Epoch 3/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - accuracy: 0.6037 - loss: 0.6559 - val_accuracy: 0.4787 - val_loss: 0.7120
Epoch 4/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step - accuracy: 0.6329 - loss: 0.6359 - val_accuracy: 0.4620 - val_loss: 0.7306
Epoch 5/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step - accuracy: 0.6413 - loss: 0.6213 - val_accuracy: 0.4603 - val_loss: 0.7696
Epoch 6/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step - accuracy: 0.6654 - loss: 0.5999 - val_accuracy: 0.4620 - val_loss: 0.7871
Epoch 7/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - accuracy: 0.6716 - loss: 0.5853 - val_accuracy: 0.4573 - val_loss: 0.8416
Epoch 8/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - accuracy: 0.6794 - loss: 0.5710 - 